In [1]:
!pip install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.8 MB/s eta 0:00:00


In [2]:
from reportlab.lib.pagesizes import letter
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen.canvas import Canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib import colors
from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet
from reportlab.platypus import Paragraph
import pandas as pd
import math
import re

In [3]:
pdfmetrics.registerFont(TTFont('Kalam-Bold', 'Kalam-Bold.ttf'))
pdfmetrics.registerFont(TTFont('Kalam-Light', 'Kalam-Light.ttf'))
pdfmetrics.registerFont(TTFont('Poppins-Bold', 'Poppins-Bold.ttf'))
#pdfmetrics.registerFont(TTFont('Black_Bones', '/content/BlackBones.ttf'))
pdfmetrics.registerFont(TTFont('IBMPlexSans-Bold', 'IBMPlexSans-Bold.ttf'))

# Elemental Functions
### General parameter style- (canvas,object,coordinate,dimension,coloring,etc_styling)

In [4]:
def insert_image(canvas, image_path, x, y, width=None, height=None):
    # Insert the image at the specified position
    if width is not None and height is not None:
        canvas.drawImage(image_path, x, y, width=width, height=height, mask='auto')
    else:
        canvas.drawImage(image_path, x, y, mask='auto')

In [5]:
def add_text(canvas, text, x, y, font_size=12, font_style="Helvetica",color=(0, 0, 0), underline=False, italic=False, bold=False):
    # Set font size and font type
    font_name = font_style
    if bold:
        font_name += "-Bold"
    if italic:
        font_name += "-Oblique"

    canvas.setFont(font_name, font_size)
    canvas.setStrokeColorRGB(*color)

    # Set text decoration (underline)
    if underline:
        canvas.drawString(x, y - font_size - 1, text, mode="underline")
    else:
        canvas.drawString(x, y - font_size - 1, text)

In [6]:
def add_line(canvas, x, y, length, width=1, color=(0, 0, 0), line_style=None):
    # Calculate start and end points based on center coordinates and length
    x1 = x - length / 2
    x2 = x + length / 2
    y1 = y2 = y

    # Set line attributes
    canvas.setLineWidth(width)
    canvas.setStrokeColorRGB(*color)

    if line_style == 'dotted':
        canvas.setDash([1, 2], 0)
    elif line_style == 'dashed':
        canvas.setDash([4, 2], 0)

    # Draw the line
    canvas.line(x1, y1, x2, y2)

In [7]:
def add_rounded_text_box(canvas, text, x, y, corner_radii=10, text_size=12, text_font="Helvetica", bg_color=(0, 0, 0), border_color=(0, 0, 0), text_color=(0, 0, 0)):
    # Calculate text width and height
    text_width = canvas.stringWidth(text, text_font, text_size)
    text_height = text_size * 1.2

    # Calculate box size with margins
    box_width = text_width + 10
    box_height = text_height + 10

    # Set border color and background color
    border_color = tuple(val / 255 for val in border_color)
    bg_color = tuple(val / 255 for val in bg_color)

    canvas.setStrokeColorRGB(*border_color)
    canvas.setFillColorRGB(*bg_color)

    # Set border thickness
    canvas.setLineWidth(2)

    # Draw rounded rectangle
    canvas.roundRect(x, y, box_width, box_height, corner_radii, stroke=1, fill=1)

    # Set text color and font
    canvas.setFillColorRGB(*text_color)
    canvas.setFont(text_font, text_size)

    # Draw the text at the center of the box
    text_x = x + (box_width - text_width) / 2
    text_y = y + (box_height - text_height) / 2 + 7
    canvas.drawString(text_x, text_y, text)

In [8]:
def add_rounded_box(canvas, x, y, width, height, radius=10,border_thickness=1, border_color=(0, 0, 0), bg_color=(255, 255, 255)):
    # Convert RGB values to floats in the range 0 to 1
    border_color = tuple(val / 255 for val in border_color)
    bg_color = tuple(val / 255 for val in bg_color)

    # Set border color and background color
    canvas.setStrokeColorRGB(*border_color)
    canvas.setFillColorRGB(*bg_color)

    # Set border thickness
    canvas.setLineWidth(border_thickness)

    # Draw rounded rectangle
    canvas.roundRect(x, y, width, height, radius, stroke=1, fill=1)

## Score Viz

In [9]:
def score_viz(score):
  insert_image(canvas, '/content/btmLeft.png', 10, 275, 250,75 )
  add_rounded_text_boxw(canvas, "Your Score- ", 70,305, corner_radii=4, bg_color=(255,255,255), border_color=(255,255, 255), text_color=(0, 0, 0), text_size=16, text_font="Kalam-Bold")
  add_rounded_text_boxw(canvas, score, 160,300, corner_radii=4, bg_color=(255,255,255), border_color=(255,255, 255), text_color=(200, 0, 0), text_size=28, text_font="Kalam-Bold")


# Compound Functions

In [10]:
#@title Letter head
def let_head(canvas, name, cls, roll, sec):
    # Define image parameters
    image_path_1 = '/content/testImg.jpeg'  # Replace with the actual path to your image
    image_path_2 = '/content/compImg.jpeg'

    x1, y1 = A4[0] - 590, A4[1] - 65  # Position at the top-right corner
    x2, y2 = A4[0] - 155, A4[1] - 50  # Position for the second image

    # Call the function to insert images
    insert_image(canvas, image_path_1, x1, y1, 166, 53)
    insert_image(canvas, image_path_2, x2, y2, 135, 27)

    # Use placeholder formatting for side-by-side text
    formatted_text = "{} | Class {} | Roll NO. {} | Sec {}".format(name, cls, roll, sec)

    # Calculate the width of the text
    # text_width = canvas.stringWidth(name, "IBMPlexSans-Bold", 11) + \
    #              canvas.stringWidth(cls, "IBMPlexSans-Bold", 11) + \
    #              canvas.stringWidth(roll, "IBMPlexSans-Bold", 11) + \
    #              canvas.stringWidth(sec, "IBMPlexSans-Bold", 11)

    text_width = canvas.stringWidth(formatted_text, "IBMPlexSans-Bold", 11)

    x = (A4[0] - text_width) / 2  # Center the text with respect to x-axis
    y = A4[1] - 25  # Adjust as needed



    add_text(canvas, formatted_text, x, y, font_size=11, font_style="IBMPlexSans-Bold")

    ##solid,dotted,dashed
    add_line(canvas, 300, 760, length=625, width=1, color=(0, 0, 0), line_style='solid')


In [11]:

def add_rounded_text_boxm(canvas,text ,x, y, box_width, box_height, radii,box_border_thickness, box_border_color, box_bg_color,
                         text_size, text_color, text_font, text_to_box_padding):
    # Draw the box
    canvas.setStrokeColorRGB(*box_border_color)
    canvas.setFillColorRGB(*box_bg_color)
    canvas.setLineWidth(box_border_thickness)
    canvas.roundRect(x, y, box_width, box_height, radii, stroke=1, fill=1)

    # Set font size and font type for the text
    canvas.setFont(text_font, text_size)

    # Calculate the available width and height for the text
    text_width = box_width - 2 * text_to_box_padding
    text_height = box_height - 2 * text_to_box_padding

    # Split the text into lines that fit within the available width
    lines = []
    words = text.split()
    line = words[0]
    for word in words[1:]:
        if canvas.stringWidth(line + ' ' + word, text_font, text_size) <= text_width:
            line += ' ' + word
        else:
            lines.append(line)
            line = word
    lines.append(line)

    # Draw the text lines within the box
    text_x = x + text_to_box_padding
    text_y = y + box_height - text_to_box_padding - text_size
    for line in lines:
        canvas.setFillColorRGB(*text_color)
        canvas.drawString(text_x, text_y, line)
        text_y -= text_size * 1.2  # Move to the next line


In [12]:
def add_rounded_text_boxw(canvas, text, x, y, corner_radii=10, text_size=12, text_font="Helvetica", bg_color=(0, 0, 0), border_color=(0, 0, 0), text_color=(0, 0, 0)):
    # Calculate text width and height
    text_width = canvas.stringWidth(text, text_font, text_size)
    text_height = text_size * 1.2

    # Calculate box size with margins
    box_width = text_width
    box_height = text_height

    # Set border color and background color
    border_color = tuple(val / 255 for val in border_color)
    bg_color = tuple(val / 255 for val in bg_color)

    canvas.setStrokeColorRGB(*border_color)
    canvas.setFillColorRGB(*bg_color)

    # Set border thickness
    canvas.setLineWidth(2)

    # Draw rounded rectangle
    canvas.roundRect(x, y, box_width, box_height, corner_radii, stroke=1, fill=1)

    # Set text color and font
    canvas.setFillColorRGB(*text_color)
    canvas.setFont(text_font, text_size)

    # Draw the text at the center of the box
    text_x = x + (box_width - text_width) / 2
    text_y = y + (box_height - text_height) / 2 + 3
    canvas.drawString(text_x, text_y, text)

In [13]:
def add_rounded_boxwm(canvas, text,x, y, radii, box_width, box_height, bg_color, border_color, text_color, text_size, text_font,bold=False, italic=False):
    # Calculate the width and height of the rounded box based on the provided dimensions
    margin = 2
    width = box_width - 2 * margin
    height = box_height - 2 * margin

    # Set border color and background color
    canvas.setStrokeColorRGB(*border_color)
    canvas.setFillColorRGB(*bg_color)

    # Draw rounded rectangle
    canvas.roundRect(x, y, width, height, radii, stroke=1, fill=1)

    # Set text color and font
    canvas.setFillColorRGB(*text_color)

    if bold:
        text_font += "-Bold"
    if italic:
        text_font += "-Italic"

    canvas.setFont(text_font, text_size)

    # Calculate text position
    x_text = x + margin
    y_text = y + height - margin -7  # Start from the top

    # Draw the multiline text within the box
    words = text.split()
    lines = []
    current_line = words[0]

    for word in words[1:]:
        test_line = current_line + ' ' + word
        if canvas.stringWidth(test_line, text_font, text_size) <= width:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word

    # Add the last line
    lines.append(current_line)

    # Draw the lines
    for line in lines:
        canvas.drawString(x_text, y_text, line)
        y_text -= text_size + 2  # Move to the next line


In [14]:
def add_rounded_text_boxr(canvas, x, y, width, height, corner_radius, border_color, bg_color, border_thickness, text_size, text_color, text_font, text):
    # Set border color and background color
    border_color = tuple(val / 255 for val in border_color)
    bg_color = tuple(val / 255 for val in bg_color)

    canvas.setStrokeColorRGB(*border_color)
    canvas.setFillColorRGB(*bg_color)

    # Set border thickness
    canvas.setLineWidth(border_thickness)

    # Draw rounded rectangle
    canvas.roundRect(x, y, width, height, corner_radius, stroke=1, fill=1)

    # Split the multiline text into separate lines and reverse the order
    lines = text.split('\n')[::-1]

    # Set text color and font
    canvas.setFillColorRGB(*text_color)
    canvas.setFont(text_font, text_size)

    # Calculate the total height of all lines
    total_text_height = len(lines) * text_size * 1.2

    # Draw each line of text
    for line in lines:
        text_width = canvas.stringWidth(line, text_font, text_size)
        text_x = x + 10  # Left justify by adding a small margin
        text_y = y + (height - total_text_height) + text_size * 0.2
        canvas.drawString(text_x, text_y, line)
        total_text_height -= text_size * 1.2



In [15]:
def add_rounded_text_boxt(canvas, x, y, width, height, corner_radius, border_color, bg_color, border_thickness, max_text_size, text_color, text_font, text):

    # Set border color and background color
    border_color = tuple(val / 255 for val in border_color)
    bg_color = tuple(val / 255 for val in bg_color)

    canvas.setStrokeColorRGB(*border_color)
    canvas.setFillColorRGB(*bg_color)

    # Set border thickness
    canvas.setLineWidth(border_thickness)

    # Draw rounded rectangle
    canvas.roundRect(x, y, width, height, corner_radius, stroke=1, fill=1)

    # Calculate text area dimensions
    text_x = x + 10
    text_width = width - 20

    # Initialize text size
    text_size = max_text_size

    # Try reducing text size until the text fits within the text area
    while text_size > 0:
        # Set text color and font
        canvas.setFillColorRGB(*text_color)
        canvas.setFont(text_font, text_size)

        # Split the multi-line text into lines that fit within the text area
        lines = []
        line = ""
        for word in text.split():
            if canvas.stringWidth(line + word, text_font, text_size) <= (text_width + 30):
                line += word + " "
            else:
                lines.insert(0, line)  # Insert at the beginning of the list to maintain order
                line = word + " "
        lines.insert(0, line)

        # Calculate total text height
        total_text_height = len(lines) * text_size * 1.2

        # Check if the text fits within the text area
        if total_text_height <= height:
            break
        else:
            # Reduce text size and try again
            text_size -= 1

    # Calculate total text height
    total_text_height = len(lines) * text_size * 1.2
    text_y = (y + (height - total_text_height) / 2)

    # Draw each line of text within the text area
    for line in lines:
        text_y += text_size #* 1.2
        canvas.drawString(text_x, text_y, line)


In [16]:
def add_text_to_box(pdf_canvas, text, x, y, box_width, box_height, font_size=12, font_style="Kalam", task_indent=0, line_spacing=1):

    from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
    from reportlab.platypus import Paragraph

    # Split the text into lines
    lines = text.split('\n')

    # Create paragraph styles for bold and regular text
    styles = getSampleStyleSheet()
    bold_style = ParagraphStyle('BoldStyle', parent=styles['Normal'], fontName='Kalam-Bold', fontSize=font_size, leading=font_size * line_spacing)
    regular_style = ParagraphStyle('RegularStyle', parent=styles['Normal'], fontName='Kalam-Light', fontSize=font_size, leading=font_size * line_spacing)

    # Calculate the available space in the box
    available_height = box_height - 10  # Margin

    # Initialize the y-coordinate for text positioning
    y_position = y + box_height - 5  # Start from the top

    # Loop through each line of text and add it to the PDF
    for line in lines:
        if line.startswith("Task") :  # Bold for task headings
            task_heading = line
            paragraph = Paragraph(task_heading, bold_style)

            # Draw the paragraph on the canvas
            paragraph_width, paragraph_height = paragraph.wrap(box_width, available_height)
            paragraph.drawOn(pdf_canvas, x + 5, y_position - paragraph_height)

            y_position -= paragraph_height  # Move to the next line
        else:
            indented_line = " " * task_indent + line  # Indent each line by the specified amount
            paragraph = Paragraph(indented_line, regular_style)

            # Draw the paragraph on the canvas
            paragraph_width, paragraph_height = paragraph.wrap(box_width - task_indent*font_size, available_height)
            paragraph.drawOn(pdf_canvas, x + 5 + task_indent*font_size, y_position - paragraph_height)

            y_position -= paragraph_height  # Move to the next line

            # If the remaining available height is not enough for another line, stop
            if y_position < y:
                break

## Suggest Box

In [17]:
#->score will be passed as argument and resutls accordingly
def suggest_box(txt,color):
    import re

    add_rounded_text_boxr(canvas, 273,40 , 300, 323, 30, color, (255, 255, 255), 4, 7, (0, 0, 0), "Kalam-Light", "                  ")
    add_rounded_box(canvas,303,305,215,45,radius=22.5,border_color=color,bg_color=color,border_thickness=1.2)
    add_rounded_box(canvas,308,310,215,45,radius=22.5,border_color=color,bg_color=(255,255,255),border_thickness=1.2)

    match = re.search(r'("[^"]*")', txt)
    if match:
        captured_text = match.group(1)
        title=captured_text
    else:
        title = "N/A"
    add_rounded_text_boxt(canvas, 318,317 , 170, 30,2 , (255, 255, 255), (255, 255, 255), 0.5, 8, (0, 0, 0), "Kalam-Bold", title)

    # Define the regular expression pattern
    pattern = r'Task \d+: (.*?)(?=Task \d+|$)'
    # Find all matches in the text
    matches = re.findall(pattern, txt, re.DOTALL)
    # Store the captured tasks in a variable
    captured_tasks = "\n\n".join([f"Task {i}: {match.strip()}" for i, match in enumerate(matches, start=1)])

    add_text_to_box(canvas, captured_tasks, 290, 86, 250, 208, 8, task_indent=2,line_spacing=1,font_style="Kalam-Light")


In [18]:
def MeanEx_box(canvas, text, x, y, box_width, box_height, radii, box_thickness=1, text_size=12, font_style="Helvetica", box_border_color=(255,102,196), bg_color=(255,255,255), text_color=(0,0,0)):

    canvas.setLineWidth(box_thickness)
    canvas.setFillColorRGB(*[col/255 for col in box_border_color])  # Convert color values to range [0, 1]
    canvas.setStrokeColorRGB(*[col/255 for col in box_border_color])
    canvas.roundRect(x, y, box_width, box_height, radii, stroke=1, fill=0)

    # Create paragraph styles
    styles = getSampleStyleSheet()
    bold_style = ParagraphStyle('BoldStyle', parent=styles['Normal'], fontName=font_style + "-Bold", fontSize=text_size, textColor=text_color, leading=text_size * 1.2)
    regular_style = ParagraphStyle('RegularStyle', parent=styles['Normal'], fontName=font_style+"-Light", fontSize=text_size, textColor=text_color, leading=text_size * 1.2)

    # Split the text into sections
    sections = text.split('\n\n')

    # Draw the text inside the rounded box
    y_position = y + box_height - 5
    for section in sections:
        lines = section.split('\n')
        for i, line in enumerate(lines):
            if line.endswith(":"):
                p = Paragraph(line, bold_style)
            else:
                p = Paragraph(line, regular_style)

            p.wrapOn(canvas, box_width - 20, box_height - 20)
            p.drawOn(canvas, x + 10, y_position - p.height)
            y_position -= p.height + 5



In [19]:
def Factor_box(canvas, text, x, y, box_width, box_height, radii, box_thickness=1, text_size=12, font_style="Helvetica", box_border_color=(255,102,196), bg_color=(255,255,255), text_color=(0,0,0)):
    canvas.setLineWidth(box_thickness)
    canvas.setFillColorRGB(*[col/255 for col in box_border_color])  # Convert color values to range [0, 1]
    canvas.setStrokeColorRGB(*[col/255 for col in box_border_color])
    canvas.roundRect(x, y, box_width, box_height, radii, stroke=1, fill=0)

    # Create paragraph styles
    styles = getSampleStyleSheet()
    bold_style = ParagraphStyle('BoldStyle', parent=styles['Normal'], fontName=font_style + "-Bold", fontSize=text_size, textColor=text_color, leading=text_size * 1.2)
    regular_style = ParagraphStyle('RegularStyle', parent=styles['Normal'], fontName=font_style+"-Light", fontSize=text_size, textColor=text_color, leading=text_size * 1.2)

    # Split the text into sections
    sections = text.split('\n\n')

    # Draw the text inside the rounded box
    y_position = y + box_height - 10
    for section in sections:
        lines = section.split('\n')
        for i, line in enumerate(lines):
            if i == 0:
                p = Paragraph(line, bold_style)
            else:
                p = Paragraph(line, regular_style)

            p.wrapOn(canvas, box_width - 20, box_height - 20)
            p.drawOn(canvas, x + 10, y_position - p.height)
            y_position -= p.height + 5


# Complete Report

In [28]:
# Create a canvas
output_filename = "BT-1.pdf"
canvas = Canvas(output_filename, pagesize=A4)

###<<<<<<<<<<<<<<<<<<<<--------------------------->>>>>>>>>>>>>>>>>>>>>>>>

def fr(name,cls=6,sec="A",roll=1,Trait="LOGIC",color=(203,108,230),trait_tile=False):

  data= pd.read_csv("/content/BPNB-Class6.csv")
  df=pd.read_csv("/content/BPNB-Class"+str(cls)+".csv")

  for i in range(len(df)):
    if (df.NAME[i] == name) & (df.SECTION[i] == sec) & (df.ROLL[i] == roll):
      name=df.NAME[i]
      sec=df.SECTION[i]
      roll=df.ROLL[i]
      c=i

  tr=pd.read_excel("/content/Trait wise Statement.xlsx",skiprows=2).transpose()
  td = pd.read_excel("/content/TRAIT DESC.xlsx")


  score=df[Trait][c].round(2)
  l=["LOGIC","CONCENTRATION","FOCUS","COGNITIVE SKILL","RETENTION POWER","HARD WORKING","STUDY HABIT","CONSCIOUSNESS","SILLY MISTAKES","KNOWLEDGE GAP","IMPULSIVE","LEARNING GAP"]
  c=l.index(Trait)
  if math.ceil(score) == 0:
    ssr=tr[c][1]
  else:
    ssr=tr[c][math.ceil(score)]



  meanex=td["Meaning"][c]
  factortxt=td["Factors"][c]

  if trait_tile == "Bright":
        add_rounded_text_box(canvas, "BRIGHT TRAITS",220,710,corner_radii=7, bg_color=(6, 172, 28), border_color=(6, 172, 28), text_color=(0, 0, 0), text_size=20, text_font="Poppins-Bold")
        add_line(canvas, 298, 720, length=147, width=2, color=(0, 0, 0), line_style='solid')
  elif trait_tile == "Dark":
        add_rounded_text_box(canvas, "DARK TRAITS",220,710,corner_radii=7, bg_color=(255, 49, 49), border_color=(255, 45, 45), text_color=(0, 0, 0), text_size=20, text_font="Poppins-Bold")
        add_line(canvas, 290, 720, length=130, width=2, color=(0, 0, 0), line_style='solid')

  let_head(canvas,name,"6",str(roll),sec)

  add_rounded_text_box(canvas, Trait,27,695,corner_radii=7, bg_color=color, border_color=color, text_color=(0, 0, 0), text_size=20, text_font="Kalam-Bold")

  MeanEx_box(canvas, meanex, 12, 548, 350, 130, 10, box_thickness=2, text_size=10, font_style="Kalam", box_border_color=color, bg_color=(255,255,255), text_color=(0,0,0))
  Factor_box(canvas, factortxt, 12, 407, 350, 130, 10, box_thickness=2, text_size=10, font_style="Kalam", box_border_color=color, bg_color=(255,255,255), text_color=(0,0,0))

  add_line(canvas, 300, 395, length=580, width=1.2, color=(0, 0, 0), line_style='solid')
  add_rounded_text_box(canvas, "Your Report",10,355,corner_radii=7, bg_color=color, border_color=color, text_color=(0, 0, 0), text_size=20, text_font="Kalam-Bold")
  insert_image(canvas, '/content/'+Trait+'.png', 365, 440, 225,250 )
  add_rounded_box(canvas,15,40,250,230,radius=20,border_color=color,bg_color=(255,255,255),border_thickness=3)

  score_viz(str(score))

  suggest_box(ssr,color)
  # canvas.showPage()
  # Save the canvas
  #canvas.save()



In [30]:
name="Deboshruti Bagchi"  ##Pls type the correct details
roll=56

fr(name,cls=6,sec="A",roll=roll,Trait="LOGIC",color=(255,102,196),trait_tile="Bright")#(255,168,120)
canvas.showPage()
fr(name,cls=6,sec="A",roll=roll,Trait="CONCENTRATION",color=(82,113,255))
canvas.showPage()
fr(name,cls=6,sec="A",roll=roll,Trait="FOCUS",color=(251,186,14))
canvas.showPage()
fr(name,cls=6,sec="A",roll=roll,Trait="COGNITIVE SKILL",color=(0,191,99))
canvas.showPage()
fr(name,cls=6,sec="A",roll=roll,Trait="RETENTION POWER",color=(253,91,50))
canvas.showPage()
fr(name,cls=6,sec="A",roll=roll,Trait="HARD WORKING",color=(255,168,120))
canvas.showPage()
fr(name,cls=6,sec="A",roll=roll,Trait="STUDY HABIT",color=(255,222,89))
canvas.showPage()
fr(name,cls=6,sec="A",roll=roll,Trait="CONSCIOUSNESS",color=(203,108,230))
canvas.showPage()
fr(name,cls=6,sec="A",roll=roll,Trait="SILLY MISTAKES",color=(255,188,125),trait_tile="Dark")
canvas.showPage()
fr(name,cls=6,sec="A",roll=roll,Trait="KNOWLEDGE GAP",color=(96,207,255))
canvas.showPage()
fr(name,cls=6,sec="A",roll=roll,Trait="IMPULSIVE",color=(255,49,49))
canvas.showPage()
fr(name,cls=6,sec="A",roll=roll,Trait="LEARNING GAP",color=(28,122,239))
canvas.save()

In [22]:
# fr("Sani Sarkar",cls=6,sec="A",roll=56,Trait="SILLY MISTAKES",color=(255,49,49),trait_tile="Dark")
# canvas.save()

In [23]:
# l=["LOGIC","CONCENTRATION","FOCUS","COGNITIVE SKILL","RETENTION POWER","HARD WORKING","STUDY HABIT","CONSCIOUSNESS","SILLY MISTAKES","KNOWLEDGE GAP","IMPULSIVE","LEARNING GAP"]
# colors=[(255,102,196),(82,113,255),(251,186,14),(0,191,99),(253,91,50),(255,168,120),(255,222,89),(203,108,230),(255,188,125),(96,207,255),(255,49,49),(28,122,239)]

# def complete_Report(name,cls=6,sec="A",roll=1,canvas=canvas):
#   output_filename = "CR-1.pdf"
#   canvas = Canvas(output_filename, pagesize=A4)
#   for i in range(len(l)):
#     if l[i] == "LOGIC":
#       fr(name,cls,sec,roll,Trait="LOGIC",color=colors[i],trait_tile="Bright")

#     elif l[i] == "SILLY MISTAKES":
#       fr(name,cls,sec,roll,Trait="SILLY MISTAKES",color=colors[i],trait_tile="Dark")

#     else :
#       fr(name,cls,sec,roll,Trait=l[i],color=colors[i])

#   canvas.save()



In [24]:
# output_filename = "CR-1.pdf"
# canvas = Canvas(output_filename, pagesize=A4)
# complete_Report("Deboshruti Bagchi",6,"A",1)